In [1]:
import ee
ee.Initialize()
from iteru import *
from IPython.display import display
import ipywidgets
import datetime

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [ ]:
aoi = Map.aoi

In [ ]:
dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')\
           .filter(ee.Filter.bounds(aoi))\
           .select('DSM')\
           .median()\
           .clip(aoi)

In [ ]:
demvis = {'min':488,'max':1863,'palette':['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

In [ ]:
Map.add_layer_widgets(dataset,demvis,'GRED_DEM')

In [ ]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")\
     .filter(ee.Filter.bounds(aoi))\
     .filter(ee.Filter.date('2020-01-01','2020-02-1'))\
     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))

In [ ]:
S2.size().getInfo()

In [ ]:
vis_param = get_vis_params(S2)

In [ ]:
S2_median = S2.median().clip(aoi)

In [ ]:
Map.add_layer_widgets(S2_median,vis_param,'S2_Median')

In [ ]:
awei = S2_median.expression(
    '4*(GREEN - SWIR1) - (0.25*NIR + 2.75*SWIR2)', {
      'GREEN': S2_median.select('B3').multiply(0.0001),
      'NIR': S2_median.select('B8').multiply(0.0001),
      'SWIR1': S2_median.select('B11').multiply(0.0001),
      'SWIR2': S2_median.select('B12').multiply(0.0001),

}).rename('awei')

In [ ]:
aweimasked = awei.updateMask(awei.gte(-0.3))

In [ ]:
Map.add_layer_widgets(aweimasked,{'min':0,'max':0.5,'palette': ['00FFFF', '0000FF']},'AWEI')

In [ ]:
lake_dem = dataset.updateMask(awei.gte(-0.3))

In [ ]:
Map.add_layer_widgets(lake_dem,demvis,'Lake DEM')

In [ ]:
ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = aoi, 
                         scale = 10, 
                        maxPixels = 1e10)

In [ ]:
ele.getInfo()